# Practica 1
## Equipo 4

## Sistema de Lógica Difusa

### Configuración Inicial

In [1]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

### Definición de variables y conjuntos difusos

En esta sección, estamos definiendo las **variables difusas** que serán utilizadas en el sistema de lógica difusa para evaluar el **riesgo de impago** de una persona. Estas variables se dividen en:

1. **Antecedentes**:
   - **Ingreso mensual**: Representa el nivel de ingresos de una persona, con un rango de 0 a 10 millones de pesos.
   - **Estabilidad laboral**: Representa la estabilidad laboral de una persona, con un rango de 0 a 10 en una escala subjetiva.
   - **Puntaje crediticio**: Representa el puntaje crediticio de una persona, con un rango de 0 a 1000, basado en estándares financieros.

2. **Consecuente**:
   - **Riesgo de impago**: Representa el nivel de riesgo de que una persona no cumpla con sus obligaciones financieras, con un rango de 0 a 100 (porcentaje).

In [2]:
# Antecedentes
ingreso_mensual = ctrl.Antecedent(np.arange(0,10.1,0.1),'ingreso_mensual')
estabilidad_laboral = ctrl.Antecedent(np.arange(0,10.1,0.1),'estabilidad_laboral')
puntaje_crediticio = ctrl.Antecedent(np.arange(0,1001,1),'puntaje_crediticio')

# Consecuente
riesgo_impago = ctrl.Consequent(np.arange(0,101,1),'riesgo_impago')

A continuación, se definen las **funciones de pertenencia** para las variables difusas previamente declaradas. Estas funciones permiten modelar los valores lingüísticos asociados a cada variable, como "bajo", "medio", "alto", entre otros, utilizando diferentes formas matemáticas (trapezoidal, triangular, gaussiana, etc.).

**Detalles**:
1. **Ingreso mensual**:
   - Valores lingüísticos: "bajo", "medio", "alto".
   - Modificadores: "más o menos alto", "extremadamente alto".

2. **Estabilidad laboral**:
   - Valores lingüísticos: "inestable", "moderada", "estable".
   - Modificadores: "muy estable", "más o menos estable".

3. **Puntaje crediticio**:
   - Valores lingüísticos: "malo", "regular", "bueno".
   - Modificadores: "muy bueno", "más o menos malo".

4. **Riesgo de impago**:
   - Valores lingüísticos: "bajo", "medio", "alto".
   - Modificadores: "muy bajo", "más o menos alto".

In [ ]:
# Ingreso mensual
ingreso_mensual['bajo'] = fuzz.trapmf(ingreso_mensual.universe, [0, 0, 1.4, 2.8])
ingreso_mensual['medio'] = fuzz.trimf(ingreso_mensual.universe, [1.4, 3, 5.5])
ingreso_mensual['alto'] = fuzz.gaussmf(ingreso_mensual.universe, 10, 2.8)

# Modificadores para ingreso mensual
ingreso_mensual['mas o menos alto'] = np.sqrt(fuzz.gaussmf(ingreso_mensual.universe, 10, 2.8))
ingreso_mensual['extremadamente alto'] = np.power(fuzz.gaussmf(ingreso_mensual.universe, 10, 2.8), 3)

# Estabilidad laboral
estabilidad_laboral['inestable'] = fuzz.trapmf(estabilidad_laboral.universe, [0, 0, 2, 4])
estabilidad_laboral['moderada'] = fuzz.trimf(estabilidad_laboral.universe, [3, 5.5, 7])
estabilidad_laboral['estable'] = fuzz.gaussmf(estabilidad_laboral.universe, 10, 2.5)

# Modificadores para estabilidad laboral
estabilidad_laboral['muy estable'] = np.power(fuzz.gaussmf(estabilidad_laboral.universe, 10, 2.5), 2)
estabilidad_laboral['mas o menos estable'] = np.sqrt(fuzz.gaussmf(estabilidad_laboral.universe, 10, 2.5))

# Puntaje crediticio
puntaje_crediticio['malo'] = fuzz.trimf(puntaje_crediticio.universe, [0, 0, 500])
puntaje_crediticio['regular'] = fuzz.gaussmf(puntaje_crediticio.universe, 500, 150)
puntaje_crediticio['bueno'] = fuzz.trapmf(puntaje_crediticio.universe, [600, 720, 1000, 1000])

# Modificadores para puntaje crediticio
puntaje_crediticio['muy bueno'] = np.power(fuzz.trapmf(puntaje_crediticio.universe, [600, 800, 1000, 1000]), 2)
puntaje_crediticio['mas o menos malo'] = np.sqrt(fuzz.trimf(puntaje_crediticio.universe, [0, 250, 450]))

# Riesgo de impago
riesgo_impago['bajo'] = fuzz.trimf(riesgo_impago.universe, [0, 0, 35])
riesgo_impago['medio'] = fuzz.trapmf(riesgo_impago.universe, [20, 40, 60, 80])
riesgo_impago['alto'] = fuzz.gaussmf(riesgo_impago.universe, 100, 25)

# Modificadores para riesgo de impago
riesgo_impago['muy bajo'] = np.power(fuzz.trimf(riesgo_impago.universe, [0, 0, 35]), 2)
riesgo_impago['mas o menos alto'] = np.sqrt(fuzz.gaussmf(riesgo_impago.universe, 100, 25))

### Definición de reglas difusas

En esta sección, se definen las reglas difusas que serán utilizadas para evaluar el **riesgo de impago** de una persona, basándose en las variables de entrada: **ingreso mensual**, **estabilidad laboral** y **puntaje crediticio**.



In [19]:
# Regla 1: Si el ingreso es bajo y el puntaje es malo, entonces el riesgo es alto.
regla1 = ctrl.Rule(
    ingreso_mensual['bajo'] & puntaje_crediticio['malo'],
    riesgo_impago['alto']
)

# Regla 2: Si el ingreso es medio o la estabilidad es moderada y el puntaje es regular, entonces riesgo medio.
regla2 = ctrl.Rule(
    (ingreso_mensual['medio'] | estabilidad_laboral['moderada']) & puntaje_crediticio['regular'],
    riesgo_impago['medio']
)

# Regla 3: Si el ingreso es alto o el puntaje es bueno, entonces el riesgo es bajo.
regla3 = ctrl.Rule(
    ingreso_mensual['alto'] | puntaje_crediticio['bueno'],
    riesgo_impago['bajo']
)

# Regla 4: Si el ingreso no es alto y la estabilidad es inestable, entonces el riesgo es más o menos alto.
regla4 = ctrl.Rule(
    ~ingreso_mensual['alto'] & estabilidad_laboral['inestable'],
    riesgo_impago['mas o menos alto']
)

# Regla 5: Si el ingreso es bajo o el puntaje no es bueno, entonces el riesgo es alto.
regla5 = ctrl.Rule(
    ingreso_mensual['bajo'] | ~puntaje_crediticio['bueno'],
    riesgo_impago['alto']
)

# Regla 6: Si el ingreso es medio y el puntaje es regular, el riesgo es medio.
regla6 = ctrl.Rule(
    ingreso_mensual['medio'] & puntaje_crediticio['regular'],
    riesgo_impago['medio']
)

# Regla 7: Si la estabilidad es muy estable y el puntaje no es malo, el riesgo es bajo.
regla7 = ctrl.Rule(
    estabilidad_laboral['muy estable'] & ~puntaje_crediticio['malo'],
    riesgo_impago['bajo']
)

# Regla 8: Si la estabilidad es inestable y el puntaje es malo, el riesgo es alto.
regla8 = ctrl.Rule(
    estabilidad_laboral['inestable'] & puntaje_crediticio['malo'],
    riesgo_impago['alto']
)

# Regla 9: Si el ingreso es medio y el puntaje es bueno, el riesgo es bajo.
regla9 = ctrl.Rule(
    ingreso_mensual['medio'] & puntaje_crediticio['bueno'],
    riesgo_impago['bajo']
)

# Agregar todas las reglas al sistema de control difuso
sistema_control = ctrl.ControlSystem([
    regla1, regla2, regla3, regla4, 
    regla5, regla6, regla7, regla8, regla9
])

### Defuzzificación

In [26]:
# Defuzzificación del sistema de lógica difusa

# Crear un sistema de simulación basado en el sistema de control difuso
sistema_simulacion = ctrl.ControlSystemSimulation(sistema_control)

# Asignar valores de entrada para las variables difusas
# Ejemplo: ingreso mensual = 4.5, estabilidad laboral = 6.0, puntaje crediticio = 700
sistema_simulacion.input['ingreso_mensual'] = 3
sistema_simulacion.input['estabilidad_laboral'] = 6
sistema_simulacion.input['puntaje_crediticio'] = 700

# Ejecutar la simulación
sistema_simulacion.compute()

# Obtener el valor defuzzificado para el riesgo de impago
riesgo_impago_defuzzificado = sistema_simulacion.output['riesgo_impago']

# Mostrar el resultado
print(f"El riesgo de impago defuzzificado es: {riesgo_impago_defuzzificado:.2f}")

El riesgo de impago defuzzificado es: 33.03


## Ontología y Razonamiento Semántico

### Configuración Inicial
Instalamos las librerías y preparamos el programa.

In [6]:
%pip install rdflib owlrl

Note: you may need to restart the kernel to use updated packages.


In [7]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, FOAF, DC, XSD


# Crear grafo
g = Graph()

# Namespaces
EX = Namespace("http://practica1.org/evaluacion_creditos#")
g.bind("ex", EX)
g.bind("foaf", FOAF)
g.bind("rdfs", RDFS)

### Creación de Clases

In [8]:
# Persona y sus subclases
g.add((EX.Persona, RDF.type, RDFS.Class))

g.add((EX.Estudiante, RDF.type, RDFS.Class))
g.add((EX.Estudiante, RDFS.subClassOf, EX.Persona))

g.add((EX.Jubilado, RDF.type, RDFS.Class))
g.add((EX.Jubilado, RDFS.subClassOf, EX.Persona))

g.add((EX.Empleado, RDF.type, RDFS.Class))
g.add((EX.Empleado, RDFS.subClassOf, EX.Persona))

g.add((EX.Independiente, RDF.type, RDFS.Class))
g.add((EX.Independiente, RDFS.subClassOf, EX.Persona))

# Ingreso y sus subclases
g.add((EX.Ingreso, RDF.type, RDFS.Class))

g.add((EX.IngresoFijo, RDF.type, RDFS.Class))
g.add((EX.IngresoFijo, RDFS.subClassOf, EX.Ingreso))

g.add((EX.IngresoVariable, RDF.type, RDFS.Class))
g.add((EX.IngresoVariable, RDFS.subClassOf, EX.Ingreso))

# Crédito y sus subclases
g.add((EX.Credito, RDF.type, RDFS.Class))

g.add((EX.LibreInversion, RDF.type, RDFS.Class))
g.add((EX.LibreInversion, RDFS.subClassOf, EX.Credito))

g.add((EX.CreditoHipotecario, RDF.type, RDFS.Class))
g.add((EX.CreditoHipotecario, RDFS.subClassOf, EX.Credito))

g.add((EX.CreditoVehicular, RDF.type, RDFS.Class))
g.add((EX.CreditoVehicular, RDFS.subClassOf, EX.Credito))

# Historial y sus subclases
g.add((EX.Historial, RDF.type, RDFS.Class))

g.add((EX.PuntajeCrediticio, RDF.type, RDFS.Class))
g.add((EX.PuntajeCrediticio, RDFS.subClassOf, EX.Historial))

g.add((EX.CapacidadEndeudamiento, RDF.type, RDFS.Class))
g.add((EX.CapacidadEndeudamiento, RDFS.subClassOf, EX.Historial))

# --------- SERIALIZAR PARA VER RESULTADO ----------

print(g.serialize(format="turtle"))

@prefix ex: <http://practica1.org/evaluacion_creditos#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:CapacidadEndeudamiento a rdfs:Class ;
    rdfs:subClassOf ex:Historial .

ex:Credito a rdfs:Class .

ex:CreditoHipotecario a rdfs:Class ;
    rdfs:subClassOf ex:Credito .

ex:CreditoVehicular a rdfs:Class ;
    rdfs:subClassOf ex:Credito .

ex:Empleado a rdfs:Class ;
    rdfs:subClassOf ex:Persona .

ex:Estudiante a rdfs:Class ;
    rdfs:subClassOf ex:Persona .

ex:Historial a rdfs:Class .

ex:Independiente a rdfs:Class ;
    rdfs:subClassOf ex:Persona .

ex:Ingreso a rdfs:Class .

ex:IngresoFijo a rdfs:Class ;
    rdfs:subClassOf ex:Ingreso .

ex:IngresoVariable a rdfs:Class ;
    rdfs:subClassOf ex:Ingreso .

ex:Jubilado a rdfs:Class ;
    rdfs:subClassOf ex:Persona .

ex:LibreInversion a rdfs:Class ;
    rdfs:subClassOf ex:Credito .

ex:Persona a rdfs:Class .

ex:PuntajeCrediticio a rdfs:Class ;
    rdfs:subClassOf ex:Historial .




### Creación de Propiedades

In [9]:
# ------------ PROPIEDADES ------------

# foaf:name (nombre de la persona)



g.add((FOAF.name, RDF.type, RDF.Property))
g.add((FOAF.name, RDFS.domain, EX.Persona))
g.add((FOAF.name, RDFS.range, XSD.string))

# ex:anioNacimiento
g.add((EX.anioNacimiento, RDF.type, RDF.Property))
g.add((EX.anioNacimiento, RDFS.domain, EX.Persona))
g.add((EX.anioNacimiento, RDFS.range, XSD.gYear))

# ex:tieneIngreso
g.add((EX.tieneIngreso, RDF.type, RDF.Property))
g.add((EX.tieneIngreso, RDFS.domain, EX.Persona))
g.add((EX.tieneIngreso, RDFS.range, EX.Ingreso))

# foaf:knows (Para que aparezca en el grafo)
g.add((FOAF.knows, RDF.type, RDF.Property))
g.add((FOAF.knows, RDFS.domain, EX.Persona))
g.add((FOAF.knows, RDFS.range, EX.Persona))

# ex:referido (subproperty of foaf:knows)
g.add((EX.referido, RDF.type, RDF.Property))
g.add((EX.referido, RDFS.subPropertyOf, FOAF.knows))
g.add((EX.referido, RDFS.domain, EX.Persona))
g.add((EX.referido, RDFS.range, EX.Persona))

# ex:coSolicitante (subproperty of foaf:knows)
g.add((EX.coSolicitante, RDF.type, RDF.Property))
g.add((EX.coSolicitante, RDFS.subPropertyOf, FOAF.knows))
g.add((EX.coSolicitante, RDFS.domain, EX.Persona))
g.add((EX.coSolicitante, RDFS.range, EX.Persona))

# ex:TieneCredito
g.add((EX.tieneCredito, RDF.type, RDF.Property))
g.add((EX.tieneCredito, RDFS.domain, EX.Persona))
g.add((EX.tieneCredito, RDFS.range, EX.Credito))

# ex:tieneHistorial
g.add((EX.tieneHistorial, RDF.type, RDF.Property))
g.add((EX.tieneHistorial, RDFS.domain, EX.Persona))
g.add((EX.tieneHistorial, RDFS.range, EX.Historial))

# ex:estabilidadLaboral
g.add((EX.estabilidadLaboral, RDF.type, RDF.Property))
g.add((EX.estabilidadLaboral, RDFS.domain, EX.Persona))  # Nota: aquí es para Empleado o Independiente
g.add((EX.estabilidadLaboral, RDFS.range, XSD.string))

# ex:montoIngreso
g.add((EX.montoIngreso, RDF.type, RDF.Property))
g.add((EX.montoIngreso, RDFS.domain, EX.Ingreso))
g.add((EX.montoIngreso, RDFS.range, XSD.decimal))

# ex:monto (propiedad general)
g.add((EX.monto, RDF.type, RDF.Property))
g.add((EX.monto, RDFS.domain, EX.Credito))
g.add((EX.monto, RDFS.range, XSD.decimal))

# ex:montoSolicitado (subproperty of ex:monto)
g.add((EX.montoSolicitado, RDF.type, RDF.Property))
g.add((EX.montoSolicitado, RDFS.subPropertyOf, EX.monto))
g.add((EX.montoSolicitado, RDFS.domain, EX.Credito))
g.add((EX.montoSolicitado, RDFS.range, XSD.decimal))

# ex:montoAprobado (subproperty of ex:monto)
g.add((EX.montoAprobado, RDF.type, RDF.Property))
g.add((EX.montoAprobado, RDFS.subPropertyOf, EX.monto))
g.add((EX.montoAprobado, RDFS.domain, EX.Credito))
g.add((EX.montoAprobado, RDFS.range, XSD.decimal))

# ex:plazo
g.add((EX.plazo, RDF.type, RDF.Property))
g.add((EX.plazo, RDFS.domain, EX.Credito))
g.add((EX.plazo, RDFS.range, XSD.integer))

# ex:puntajeCrediticio
g.add((EX.puntajeCrediticio, RDF.type, RDF.Property))
g.add((EX.puntajeCrediticio, RDFS.domain, EX.Historial))
g.add((EX.puntajeCrediticio, RDFS.range, XSD.decimal))

# ex:capacidadEndeudamiento
g.add((EX.capacidadEndeudamiento, RDF.type, RDF.Property))
g.add((EX.capacidadEndeudamiento, RDFS.domain, EX.Historial))
g.add((EX.capacidadEndeudamiento, RDFS.range, XSD.decimal))

# ex:estadoCredito
g.add((EX.estadoCredito, RDF.type, RDF.Property))
g.add((EX.estadoCredito, RDFS.domain, EX.Credito))
g.add((EX.estadoCredito, RDFS.range, XSD.string))

# dc:identifier
g.add((DC.identifier, RDF.type, RDF.Property))
g.add((DC.identifier, RDFS.domain, EX.Credito))
g.add((DC.identifier, RDFS.range, XSD.string))

# ------------ SERIALIZAR PARA VER RESULTADO ------------
# print(g.serialize(format="turtle"))


<Graph identifier=N1dbab129f99a46778bbbc94febaa68b4 (<class 'rdflib.graph.Graph'>)>

### Instancias

In [10]:
# 1️⃣ Personas y subclases
persona1 = URIRef(EX.Persona3654)
persona2 = URIRef(EX.Persona6798)
persona3 = URIRef(EX.Persona786)
persona4 = URIRef(EX.Persona221)

g.add((persona1, RDF.type, EX.Persona))
g.add((persona2, RDF.type, EX.Persona))
g.add((persona3, RDF.type, EX.Persona))
g.add((persona4, RDF.type, EX.Persona))

# Estudiantes
estudiante1 = URIRef(EX.Freddy) #
estudiante2 = URIRef(EX.Jose) #
estudiante3 = URIRef(EX.Camila) #
estudiante4 = URIRef(EX.Jeremias) #

g.add((estudiante1, RDF.type, EX.Estudiante))
g.add((estudiante2, RDF.type, EX.Estudiante))
g.add((estudiante3, RDF.type, EX.Estudiante))
g.add((estudiante4, RDF.type, EX.Estudiante))

# Jubilados
jubilado1 = URIRef(EX.Rosa) #
jubilado2 = URIRef(EX.Manuel) #
jubilado3 = URIRef(EX.Lucia) #
jubilado4 = URIRef(EX.Carlos) #

g.add((jubilado1, RDF.type, EX.Jubilado))
g.add((jubilado2, RDF.type, EX.Jubilado))
g.add((jubilado3, RDF.type, EX.Jubilado))
g.add((jubilado4, RDF.type, EX.Jubilado))

# Empleados
empleado1 = URIRef(EX.Maria) #
empleado2 = URIRef(EX.Alejandro) #
empleado3 = URIRef(EX.Diana) #
empleado4 = URIRef(EX.Pedro) #

g.add((empleado1, RDF.type, EX.Empleado))
g.add((empleado2, RDF.type, EX.Empleado))
g.add((empleado3, RDF.type, EX.Empleado))
g.add((empleado4, RDF.type, EX.Empleado))

# Independientes
independiente1 = URIRef(EX.Andres) #
independiente2 = URIRef(EX.Sofia) #
independiente3 = URIRef(EX.Juan) #
independiente4 = URIRef(EX.Valentina) #

g.add((independiente1, RDF.type, EX.Independiente))
g.add((independiente2, RDF.type, EX.Independiente))
g.add((independiente3, RDF.type, EX.Independiente))
g.add((independiente4, RDF.type, EX.Independiente))

# 2️⃣ Ingreso y subclases

ingreso1 = URIRef(EX.Ingreso573)
ingreso2 = URIRef(EX.Ingreso12574893)
ingreso3 = URIRef(EX.Ingreso2389054)
ingreso4 = URIRef(EX.Ingreso567)

g.add((ingreso1, RDF.type, EX.Ingreso))
g.add((ingreso2, RDF.type, EX.Ingreso))
g.add((ingreso3, RDF.type, EX.Ingreso))
g.add((ingreso4, RDF.type, EX.Ingreso))

# Ingreso Fijo
ingreso_fijo1 = URIRef(EX.SalarioMaria) #
ingreso_fijo2 = URIRef(EX.SalarioAlejandro) #
ingreso_fijo3 = URIRef(EX.PensionRosa) #
ingreso_fijo4 = URIRef(EX.PensionManuel) #

g.add((ingreso_fijo1, RDF.type, EX.IngresoFijo))
g.add((ingreso_fijo2, RDF.type, EX.IngresoFijo))
g.add((ingreso_fijo3, RDF.type, EX.IngresoFijo))
g.add((ingreso_fijo4, RDF.type, EX.IngresoFijo))

# Ingreso Variable
ingreso_variable1 = URIRef(EX.VentasAndres) #
ingreso_variable2 = URIRef(EX.ServiciosSofia) #
ingreso_variable3 = URIRef(EX.HonorariosJuan) #
ingreso_variable4 = URIRef(EX.ProyectosValentina) #

g.add((ingreso_variable1, RDF.type, EX.IngresoVariable))
g.add((ingreso_variable2, RDF.type, EX.IngresoVariable))
g.add((ingreso_variable3, RDF.type, EX.IngresoVariable))
g.add((ingreso_variable4, RDF.type, EX.IngresoVariable))

# 3️⃣ Crédito y subclases

credito1 = URIRef(EX.Credito124)
credito2 = URIRef(EX.Credito2346)
credito3 = URIRef(EX.Credito3856)
credito4 = URIRef(EX.Credito478)

g.add((credito1, RDF.type, EX.Credito))
g.add((credito2, RDF.type, EX.Credito))
g.add((credito3, RDF.type, EX.Credito))
g.add((credito4, RDF.type, EX.Credito))

# Libre Inversión
credito_li1 = URIRef(EX.CreditoBodaJose) #
credito_li2 = URIRef(EX.CreditoMueblesCamila) #
credito_li3 = URIRef(EX.CreditoEmergenciaFreddy) #
credito_li4 = URIRef(EX.CreditoEstudiosJeremias) #

g.add((credito_li1, RDF.type, EX.LibreInversion))
g.add((credito_li2, RDF.type, EX.LibreInversion))
g.add((credito_li3, RDF.type, EX.LibreInversion))
g.add((credito_li4, RDF.type, EX.LibreInversion))

# Hipotecario
credito_h1 = URIRef(EX.CreditoCasaMaria) #
credito_h2 = URIRef(EX.CreditoCasaAlejandro) #
credito_h3 = URIRef(EX.CreditoCasaDiana) #
credito_h4 = URIRef(EX.CreditoCasaPedro) #

g.add((credito_h1, RDF.type, EX.CreditoHipotecario))
g.add((credito_h2, RDF.type, EX.CreditoHipotecario))
g.add((credito_h3, RDF.type, EX.CreditoHipotecario))
g.add((credito_h4, RDF.type, EX.CreditoHipotecario))

# Vehicular
credito_v1 = URIRef(EX.CreditoAutoRosa) #
credito_v2 = URIRef(EX.CreditoAutoManuel) #
credito_v3 = URIRef(EX.CreditoMotoLucia) #
credito_v4 = URIRef(EX.CreditoMotoCarlos) #

g.add((credito_v1, RDF.type, EX.CreditoVehicular))
g.add((credito_v2, RDF.type, EX.CreditoVehicular))
g.add((credito_v3, RDF.type, EX.CreditoVehicular))
g.add((credito_v4, RDF.type, EX.CreditoVehicular))

# 4️⃣ Historial y subclases

# PuntajeCrediticio
puntaje1 = URIRef(EX.PuntajeFreddy) #
puntaje2 = URIRef(EX.PuntajeCamila) #
puntaje3 = URIRef(EX.PuntajeJose) #
puntaje4 = URIRef(EX.PuntajeJeremias) #

g.add((puntaje1, RDF.type, EX.PuntajeCrediticio))
g.add((puntaje2, RDF.type, EX.PuntajeCrediticio))
g.add((puntaje3, RDF.type, EX.PuntajeCrediticio))
g.add((puntaje4, RDF.type, EX.PuntajeCrediticio))

# CapacidadEndeudamiento
capacidad1 = URIRef(EX.CapacidadFreddy) #
capacidad2 = URIRef(EX.CapacidadCamila) #
capacidad3 = URIRef(EX.CapacidadJose) #
capacidad4 = URIRef(EX.CapacidadJeremias) #

g.add((capacidad1, RDF.type, EX.CapacidadEndeudamiento))
g.add((capacidad2, RDF.type, EX.CapacidadEndeudamiento))
g.add((capacidad3, RDF.type, EX.CapacidadEndeudamiento))
g.add((capacidad4, RDF.type, EX.CapacidadEndeudamiento))

# ------------ SERIALIZAR PARA VER RESULTADO ------------
print(g.serialize(format="turtle"))


@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix ex: <http://practica1.org/evaluacion_creditos#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:CapacidadEndeudamiento a rdfs:Class ;
    rdfs:subClassOf ex:Historial .

ex:Credito a rdfs:Class .

ex:CreditoHipotecario a rdfs:Class ;
    rdfs:subClassOf ex:Credito .

ex:CreditoVehicular a rdfs:Class ;
    rdfs:subClassOf ex:Credito .

ex:Empleado a rdfs:Class ;
    rdfs:subClassOf ex:Persona .

ex:Estudiante a rdfs:Class ;
    rdfs:subClassOf ex:Persona .

ex:Historial a rdfs:Class .

ex:Independiente a rdfs:Class ;
    rdfs:subClassOf ex:Persona .

ex:Ingreso a rdfs:Class .

ex:IngresoFijo a rdfs:Class ;
    rdfs:subClassOf ex:Ingreso .

ex:IngresoVariable a rdfs:Class ;
    rdfs:subClassOf ex:Ingreso .

ex:Jubilado a rdfs:Class ;
    rdfs:subClassOf ex:Persona

### Razonamiento

#### Generación de Hechos Iniciales

In [11]:
# ------------ Hechos ------------
g.add((estudiante1, EX.tieneCredito, credito_li3))
g.add((estudiante1, EX.tieneHistorial, puntaje1))
g.add((estudiante1, EX.tieneHistorial, capacidad1))

g.add((estudiante2, EX.tieneCredito, credito_li1))
g.add((estudiante2, EX.tieneHistorial, puntaje3))
g.add((estudiante2, EX.tieneHistorial, capacidad3))

g.add((estudiante3, EX.tieneCredito, credito_li2))
g.add((estudiante3, EX.tieneHistorial, puntaje2))
g.add((estudiante3, EX.tieneHistorial, capacidad2))

g.add((estudiante4, EX.tieneCredito, credito_li4))
g.add((estudiante4, EX.tieneHistorial, puntaje4))
g.add((estudiante4, EX.tieneHistorial, capacidad4))


g.add((jubilado1, EX.tieneIngreso, ingreso_fijo3))
g.add((jubilado1, EX.tieneCredito, credito_v1))

g.add((jubilado2, EX.tieneIngreso, ingreso_fijo4))
g.add((jubilado2, EX.tieneCredito, credito_v2))

g.add((jubilado3, EX.tieneCredito, credito_v3))

g.add((jubilado4, EX.tieneCredito, credito_v4))


g.add((empleado1, EX.tieneIngreso, ingreso_fijo1))
g.add((empleado1, EX.tieneCredito, credito_h1))

g.add((empleado2, EX.tieneIngreso, ingreso_fijo2))
g.add((empleado2, EX.tieneCredito, credito_h2))

g.add((empleado3, EX.tieneCredito, credito_h3))

g.add((empleado4, EX.tieneCredito, credito_h4))


g.add((independiente1, EX.tieneIngreso, ingreso_variable1))

g.add((independiente2, EX.tieneIngreso, ingreso_variable2))

g.add((independiente3, EX.tieneIngreso, ingreso_variable3))

g.add((independiente4, EX.tieneIngreso, ingreso_variable4))

# ------------ SERIALIZAR PARA VER RESULTADO ------------
print(g.serialize(format="turtle"))

@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix ex: <http://practica1.org/evaluacion_creditos#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:CapacidadEndeudamiento a rdfs:Class ;
    rdfs:subClassOf ex:Historial .

ex:Credito a rdfs:Class .

ex:CreditoHipotecario a rdfs:Class ;
    rdfs:subClassOf ex:Credito .

ex:CreditoVehicular a rdfs:Class ;
    rdfs:subClassOf ex:Credito .

ex:Empleado a rdfs:Class ;
    rdfs:subClassOf ex:Persona .

ex:Estudiante a rdfs:Class ;
    rdfs:subClassOf ex:Persona .

ex:Historial a rdfs:Class .

ex:Independiente a rdfs:Class ;
    rdfs:subClassOf ex:Persona .

ex:Ingreso a rdfs:Class .

ex:IngresoFijo a rdfs:Class ;
    rdfs:subClassOf ex:Ingreso .

ex:IngresoVariable a rdfs:Class ;
    rdfs:subClassOf ex:Ingreso .

ex:Jubilado a rdfs:Class ;
    rdfs:subClassOf ex:Persona

#### Deducciones

In [12]:
from owlrl import DeductiveClosure, RDFS_Semantics

def consulta(caso):
    if caso == 1:
        for s, p, o in g.triples((EX.Freddy, RDF.type, None)):
            print(s, p, o)
        print("---")
        for s, p, o in g.triples((EX.ProyectosValentina, RDF.type, None)):
            print(s, p, o)

    elif caso == 2:
        for s, p, o in g.triples((EX.Marta, None, None)):
            print(s, p, o)
        for s, p, o in g.triples((EX.CreditoViajeMarta, None, None)):
            print(s, p, o)
            
    elif caso == 3:
        for s, p, o in g.triples((EX.Juan, None, EX.Rosa)):
            print(s, p, o)
    
    elif caso == 4:
        for s, p, o in g.triples((EX.CreditoAutoRosa, None, None)):
            if str(o) == "500000":
                print(s, p, o)

    elif caso == 5:
        for s, p, o in g.triples((EX.CreditoCasaAlejandro, RDF.type, None)):
            print(s, p, o)  

    elif caso == 6:
        for s, p, o in g.triples((EX.CapacidadJose, RDF.type, None)):
            print(s, p, o)

    elif caso == 7:
        for s, p, o in g.triples((EX.ingresoDesconocidoFreddy, RDF.type, None)):
            print(s, p, o)
        for s, p, o in g.triples((EX.Freddy, EX.tieneIngreso, None)):
            print(s, p, o)


# ------------ ANTES ------------
print("Antes de la inferencia:")

# Caso 1
print("\nCaso 1: Freddy es estudiante y ProyectosValentina es un ingreso variable")
consulta(1)

# Caso 2
credito_li5 = URIRef(EX.CreditoViajeMarta)
independiente5 = URIRef(EX.Marta)
g.add((independiente5, EX.tieneCredito, credito_li5))

print("\nCaso 2: Marta tiene un crédito CreditoViajeMarta pero no se saben sus tipos")
consulta(2)

# Caso 3
g.add((EX.Juan, EX.referido, EX.Rosa))

print("\nCaso 3: Juan tiene como referido a Rosa")
consulta(3)

# 4 Casos adicionales
print("\nCaso adicional 1: CreditoAutoRosa tiene un monto solicitado de 500000")
g.add((credito_v1, EX.montoSolicitado, Literal(500000, datatype=XSD.decimal)))
consulta(4)

print("\nCaso adicional 2: CreditoCasaAlejandro es un CreditoHipotecario solamente")
consulta(5)

print("\nCaso adicional 3: CapacidadJose es una CapacidadEndeudamiento solamente")
consulta(6)

print("\nCaso adicional 4: Freddy tiene un ingreso desconocido del que no se especifica su tipo")
ingreso_desconocido = URIRef(EX.ingresoDesconocidoFreddy)
g.add((estudiante1, EX.tieneIngreso, ingreso_desconocido))
consulta(7)

print("\n****************************************************************************")
#------------ DESPUÉS ------------
DeductiveClosure(RDFS_Semantics, axiomatic_triples=True, datatype_axioms=False).expand(g)
print("\nDespués de la inferencia:")

print("\nCaso 1: Ahora se sabe que Freddy es Persona y ProyectosValentina es un Ingreso")
consulta(1)

print("\nCaso 2: Ahora se sabe que Marta es Persona y CreditoViajeMarta es un Credito")
consulta(2)

print("\nCaso 3: Ahora se sabe que Juan knows (conoce) a Rosa")
consulta(3)

# 4 Casos adicionales
print("\nCaso adicional 1: CreditoAutoRosa tiene un monto de 500000m, por jerarquía de propiedades (Caso 3)") 
consulta(4)
print("\nCaso adicional 2: CreditoCasaAlejandro es ahora también de tipo Credito (Caso 1)")
consulta(5)
print("\nCaso adicional 3: CapacidadJose es ahora también de tipo Historial (Caso 1)")
consulta(6)
print("\nCaso adicional 4: Ingreso desconocido de Freddy ahora es de tipo Ingreso (Caso 2)")
consulta(7)

Antes de la inferencia:

Caso 1: Freddy es estudiante y ProyectosValentina es un ingreso variable
http://practica1.org/evaluacion_creditos#Freddy http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://practica1.org/evaluacion_creditos#Estudiante
---
http://practica1.org/evaluacion_creditos#ProyectosValentina http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://practica1.org/evaluacion_creditos#IngresoVariable

Caso 2: Marta tiene un crédito CreditoViajeMarta pero no se saben sus tipos
http://practica1.org/evaluacion_creditos#Marta http://practica1.org/evaluacion_creditos#tieneCredito http://practica1.org/evaluacion_creditos#CreditoViajeMarta

Caso 3: Juan tiene como referido a Rosa
http://practica1.org/evaluacion_creditos#Juan http://practica1.org/evaluacion_creditos#referido http://practica1.org/evaluacion_creditos#Rosa

Caso adicional 1: CreditoAutoRosa tiene un monto solicitado de 500000
http://practica1.org/evaluacion_creditos#CreditoAutoRosa http://practica1.org/evaluacion_credi

In [13]:
# ------------ SERIALIZAR PARA COMPARAR RESULTADO ------------
print(g.serialize(format="turtle"))

@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix ex: <http://practica1.org/evaluacion_creditos#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:CapacidadEndeudamiento a rdfs:Class,
        rdfs:Resource ;
    rdfs:subClassOf ex:CapacidadEndeudamiento,
        ex:Historial,
        rdfs:Resource .

ex:Credito a rdfs:Class,
        rdfs:Resource ;
    rdfs:subClassOf ex:Credito,
        rdfs:Resource .

ex:CreditoHipotecario a rdfs:Class,
        rdfs:Resource ;
    rdfs:subClassOf ex:Credito,
        ex:CreditoHipotecario,
        rdfs:Resource .

ex:CreditoVehicular a rdfs:Class,
        rdfs:Resource ;
    rdfs:subClassOf ex:Credito,
        ex:CreditoVehicular,
        rdfs:Resource .

ex:Empleado a rdfs:Class,
        rdfs:Resource ;
    rdfs:subClassOf ex:Empleado,
        ex:Persona,
        rdfs:Resourc

# Sistema Experta

## Reglas Sistema Experta 

### Reglas Prioridad Alta

- **Regla 1**: Si la persona es menor de edad, se rechaza automaticamente el credito.
- **Regla 2**: Si la persona tiene tiene riesgo de impago > 80%, se rechaza automaticamente el credito.
- **Regla 3**: Si la persona tiene tiene un puntaje credito < 300, se rechaza automaticamente el credito.

In [27]:
from experta import *

AttributeError: module 'collections' has no attribute 'Mapping'

In [ ]:
class PersonaFact(Fact):
    """Hechos extraídos de la ontología sobre la persona."""
    name = Field(str)
    anio_nacimiento = Field(int)
    riesgo_impago = Field(float) # Resultado logica Difusa

class IngresoFact(Fact):
    """Hechos de ingreso."""
    tipo = Field(str)
    monto = Field(float)

class HistorialFact(Fact):
    """Hechos de historial crediticio."""
    puntaje = Field(float) # [0, 1000]
    capacidad_endeudamiento = Field(float)

class CreditoFact(Fact):
    """Hechos de crédito a evaluar."""
    solicitado = Field(float)
    plazo = Field(int)

class ResultadoFact(Fact):
    aprobado = Field(str)
    interes = Field(float)
